This notebook is to load PL2 data basic info and to fill in the missing spikeID and unitID in the pkl file preloaded from matlab.


In [1]:
import os
from os.path import join
import re
import math
import pickle as pkl
import time
import numpy as np
import pandas as pd
from easydict import EasyDict as edict
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from neuro_data_analysis.PL2_reading_lib import read_channel_unit_info, pl2_info
from neuro_data_analysis.mat_data_translate_lib import h5_to_dict_simplify, print_hdf5_info

# set the pandas display options width
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


mat_root = r"S:\Data-Ephys-MAT"
pkl_root = r"S:\Data-Ephys-PKL"
exp_record_pathdict = {"Alfa": r"S:\Exp_Record_Alfa.xlsx", 
                       "Beto": r"S:\ExpSpecTable_Augment.xlsx",
                       "Caos": r"S:\Exp_Record_Caos.xlsx",
                       "Diablito": r"S:\Exp_Record_Diablito.xlsx"}

c:\Users\ponce\.conda\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Use PL2 files to update the pkl files 

In [2]:
from neuro_data_analysis.PL2_reading_lib import read_channel_unit_info, pl2_info

In [3]:
ExpRecord_Hessian_All = pd.read_csv(r"ExpRecord_BigGAN_Hessian_tuning_ABCD_w_meta.csv")
ExpRecord_Evolution_All = pd.read_csv(r"ExpRecord_BigGAN_Hessian_Evol_ABCD_w_meta.csv")


### Check existance of PL2 files on local and network drives

In [4]:
PL2_roots = [r"S:\Data-Ephys-Raw", r"N:\Data-Ephys-Raw",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2020",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2021"]
for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):   
    PL2_path = None
    for root in PL2_roots:
        potential_path = join(root, f"{exprow.ephysFN}.pl2")
        if os.path.exists(potential_path):
            PL2_path = potential_path
            break
    if PL2_path is None:
        print(f"PL2 file does not exist in any of the provided roots: {exprow.ephysFN}.pl2")
        continue


47it [00:00, 181.10it/s]

PL2 file does not exist in any of the provided roots: Alfa-27082020-003.pl2


89it [00:00, 171.80it/s]

PL2 file does not exist in any of the provided roots: Alfa-27082020-002.pl2


107it [00:00, 175.56it/s]


In [6]:
print(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).query("ephysFN == 'Alfa-27082020-002'").iloc[0])


ChanQual                                                        NaN
Exp_collection                            BigGAN_FC6;BigGAN_Hessian
Expi                                                            6.0
WebCam                                                          NaN
comments          CH 7 (-1.5, -1.5) 3 1 CMAES hash 1/5. Fc6 vs b...
ephysFN                                           Alfa-27082020-002
expControlFN                         200827_Alfa_generate_BigGAN(1)
pref_chan                                                       7.0
pref_unit                                                       1.0
stim_size                                                       3.0
stimuli           N:\Stimuli\2020-BigGAN\2020-08-27-Alfa-01\2020...
Animal                                                         Alfa
Name: 17, dtype: object


In [5]:
pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).query("ephysFN == 'Alfa-27082020-003'").iloc[0]

ChanQual                                                        NaN
Exp_collection                                       BigGAN_Hessian
Expi                                                            6.0
WebCam                                                          NaN
comments          Selectivity start at 1018. Hess iamges from CH...
ephysFN                                           Alfa-27082020-003
expControlFN                          200827_Alfa_selectivity_basic
pref_chan                                                       7.0
pref_unit                                                       1.0
stim_size                                                       3.0
stimuli             N:\Stimuli\2020-Hessian\alfa-200827-selectivity
Animal                                                         Alfa
Name: 18, dtype: object

In [ ]:
print(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).query("ephysFN == 'Alfa-27082020-002'").iloc[0].stimuli)


N:\Stimuli\2020-BigGAN\2020-08-27-Alfa-01\2020-08-27-09-41-28


### Check if the spike ID and unit ID match between PL2 and pkl files

#### Find the mismatch list

In [29]:
PL2_roots = [r"S:\Data-Ephys-Raw", r"N:\Data-Ephys-Raw",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2020",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2021"]
mismatch_list = []
for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):   
    PL2_path = None
    for root in PL2_roots:
        potential_path = join(root, f"{exprow.ephysFN}.pl2")
        if os.path.exists(potential_path):
            PL2_path = potential_path
            break
    if PL2_path is None:
        print(f"PL2 file does not exist in any of the provided roots: {exprow.ephysFN}.pl2")
        continue
    spike_channel_id, spike_unit_id, spike_total_cnt = read_channel_unit_info(PL2_path, threshold=10)
    pkldata = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    if "spikeID" in pkldata["meta"].keys():
        channel_id_pkl = pkldata["meta"]["spikeID"][0].astype(int)
        length_match = len(channel_id_pkl) == len(spike_channel_id)
        exact_match = (channel_id_pkl == spike_channel_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Spike ID mismatch: PKL channel id [len: {len(channel_id_pkl)}] !=  PL2 channel id [len: {len(spike_channel_id)}]")
            # break
            mismatch_list.append(exprow.ephysFN)
    # else:
    #     pkldata["meta"]["spikeID"] = spike_channel_id[None, :]
        
    if "unitID" in pkldata["meta"].keys():
        unit_id_pkl = pkldata["meta"]["unitID"][0].astype(int)
        length_match = len(unit_id_pkl) == len(spike_unit_id)
        exact_match = (unit_id_pkl == spike_unit_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Unit ID mismatch: PKL unit id [len: {len(unit_id_pkl)}] !=  PL2 unit id [len: {len(spike_unit_id)}]")

18it [00:04,  4.93it/s]

PL2 file does not exist in any of the provided roots: Alfa-27082020-003.pl2


72it [00:17,  3.24it/s]

PL2 file does not exist in any of the provided roots: Alfa-27082020-002.pl2


85it [00:20,  2.90it/s]C:\Users\ponce\AppData\Local\Temp\ipykernel_71180\2399960292.py:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  exact_match = (channel_id_pkl.astype(int) == spike_channel_id)
C:\Users\ponce\AppData\Local\Temp\ipykernel_71180\2399960292.py:31: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  exact_match = (unit_id_pkl.astype(int) == spike_unit_id)
86it [00:21,  2.57it/s]

...

92it [00:23,  2.93it/s]

Alfa-08042021-002 Spike ID mismatch: PKL channel id [len: 136] !=  PL2 channel id [len: 118]
Alfa-08042021-002 Unit ID mismatch: PKL unit id [len: 136] !=  PL2 unit id [len: 118]


93it [00:23,  2.99it/s]

Alfa-14052021-003 Spike ID mismatch: PKL channel id [len: 136] !=  PL2 channel id [len: 127]
Alfa-14052021-003 Unit ID mismatch: PKL unit id [len: 136] !=  PL2 unit id [len: 127]


107it [00:28,  3.79it/s]


#### Check if adaptive threshold is the reason for the mismatch

In [30]:
mismatch_list = ['Alfa-13012021-002',
 'Alfa-19022021-003',
 'Alfa-25022021-002',
 'Alfa-26022021-003',
 'Alfa-12032021-004',
 'Alfa-01042021-002',
 'Alfa-08042021-002',
 'Alfa-14052021-003']
PL2_roots = [r"S:\Data-Ephys-Raw", r"N:\Data-Ephys-Raw",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2020",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2021"]

for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):   
    PL2_path = None
    for root in PL2_roots:
        potential_path = join(root, f"{exprow.ephysFN}.pl2")
        if os.path.exists(potential_path):
            PL2_path = potential_path
            break
    if PL2_path is None:
        print(f"PL2 file does not exist in any of the provided roots: {exprow.ephysFN}.pl2")
        continue
    if exprow.ephysFN in mismatch_list:
        threshold = 1
    else:
        threshold = 10
    spike_channel_id, spike_unit_id, spike_total_cnt = read_channel_unit_info(PL2_path, threshold=threshold)
    pkldata = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    if "spikeID" in pkldata["meta"].keys():
        channel_id_pkl = pkldata["meta"]["spikeID"][0].astype(int)
        length_match = len(channel_id_pkl) == len(spike_channel_id)
        exact_match = (channel_id_pkl == spike_channel_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Spike ID mismatch: PKL channel id [len: {len(channel_id_pkl)}] !=  PL2 channel id [len: {len(spike_channel_id)}]")
        
    if "unitID" in pkldata["meta"].keys():
        unit_id_pkl = pkldata["meta"]["unitID"][0].astype(int)
        length_match = len(unit_id_pkl) == len(spike_unit_id)
        exact_match = (unit_id_pkl == spike_unit_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Unit ID mismatch: PKL unit id [len: {len(unit_id_pkl)}] !=  PL2 unit id [len: {len(spike_unit_id)}]")

['Alfa-13012021-002',
 'Alfa-19022021-003',
 'Alfa-25022021-002',
 'Alfa-26022021-003',
 'Alfa-12032021-004',
 'Alfa-01042021-002',
 'Alfa-08042021-002',
 'Alfa-14052021-003']

#### Finally write in the pkl file

In [32]:
PL2_roots = [r"S:\Data-Ephys-Raw", r"N:\Data-Ephys-Raw",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2020",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2021"]
mismatch_list = ['Alfa-13012021-002',
                'Alfa-19022021-003',
                'Alfa-25022021-002',
                'Alfa-26022021-003',
                'Alfa-12032021-004',
                'Alfa-01042021-002',
                'Alfa-08042021-002',
                'Alfa-14052021-003']
for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):   
    PL2_path = None
    for root in PL2_roots:
        potential_path = join(root, f"{exprow.ephysFN}.pl2")
        if os.path.exists(potential_path):
            PL2_path = potential_path
            break
    if PL2_path is None:
        print(f"PL2 file does not exist in any of the provided roots: {exprow.ephysFN}.pl2")
        continue
    if exprow.ephysFN in mismatch_list:
        threshold = 1
    else:
        threshold = 10
    spike_channel_id, spike_unit_id, spike_total_cnt = read_channel_unit_info(PL2_path, threshold=threshold)
    pkldata = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    if "spikeID" in pkldata["meta"].keys():
        channel_id_pkl = pkldata["meta"]["spikeID"][0].astype(int)
        length_match = len(channel_id_pkl) == len(spike_channel_id)
        exact_match = (channel_id_pkl == spike_channel_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Spike ID mismatch: PKL channel id [len: {len(channel_id_pkl)}] !=  PL2 channel id [len: {len(spike_channel_id)}]")
            break
    else:
        pkldata["meta"]["spikeID"] = spike_channel_id[None, :]
        
    if "unitID" in pkldata["meta"].keys():
        unit_id_pkl = pkldata["meta"]["unitID"][0].astype(int)
        length_match = len(unit_id_pkl) == len(spike_unit_id)
        exact_match = (unit_id_pkl == spike_unit_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Unit ID mismatch: PKL unit id [len: {len(unit_id_pkl)}] !=  PL2 unit id [len: {len(spike_unit_id)}]")
            break
    else:
        pkldata["meta"]["unitID"] = spike_unit_id[None, :]
    pkldata["meta"]["total_spike_cnt"] = spike_total_cnt
    pkl.dump(pkldata, open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "wb"))


18it [00:31,  1.76s/it]

PL2 file does not exist in any of the provided roots: Alfa-27082020-003.pl2


72it [02:43,  2.58s/it]

PL2 file does not exist in any of the provided roots: Alfa-27082020-002.pl2


107it [04:25,  2.48s/it]


#### Check if mismatch happens whether it is because of the threshold

In [26]:
PL2_path = r"S:\Data-Ephys-Raw\Alfa-27082020-002.pl2"
pkldata = pkl.load(open(join(pkl_root, f"Alfa-27082020-002.pkl"), "rb"))
spike_channel_id, spike_unit_id, spike_total_cnt = read_channel_unit_info(PL2_path, threshold=1)
print(len(spike_channel_id[spike_total_cnt > 1]))


137


In [28]:
if "spikeID" in pkldata["meta"].keys():
    channel_id_pkl = pkldata["meta"]["spikeID"][0]
    length_match = len(channel_id_pkl) == len(spike_channel_id)
    exact_match = (channel_id_pkl.astype(int) == spike_channel_id)
    if not (length_match and exact_match.all()):
        print(f"{exprow.ephysFN} Spike ID mismatch: PKL channel id [len: {len(channel_id_pkl)}] !=  PL2 channel id [len: {len(spike_channel_id)}]")
# else:
#     pkldata["meta"]["spikeID"] = spike_channel_id[None, :]

if "unitID" in pkldata["meta"].keys():
    unit_id_pkl = pkldata["meta"]["unitID"][0]
    length_match = len(unit_id_pkl) == len(spike_unit_id)
    exact_match = (unit_id_pkl.astype(int) == spike_unit_id)
    if not (length_match and exact_match.all()):
        print(f"{exprow.ephysFN} Unit ID mismatch: PKL unit id [len: {len(unit_id_pkl)}] !=  PL2 unit id [len: {len(spike_unit_id)}]")

In [18]:
channel_id_pkl

array([ 1.,  1.,  2.,  2.,  3.,  3.,  4.,  4.,  5.,  5.,  6.,  6.,  6.,
        6.,  7.,  7.,  8.,  8.,  9.,  9.,  9., 10., 10., 11., 11., 12.,
       12., 13., 13., 14., 14., 15., 15., 16., 16., 17., 17., 18., 18.,
       19., 19., 20., 20., 20., 21., 21., 22., 22., 23., 23., 24., 24.,
       24., 25., 25., 26., 26., 27., 27., 28., 28., 28., 29., 29., 30.,
       30., 31., 31., 32., 32., 32., 33., 33., 34., 34., 35., 35., 36.,
       36., 37., 37., 38., 38., 39., 39., 40., 40., 41., 41., 42., 42.,
       43., 43., 44., 44., 45., 45., 46., 46., 47., 47., 48., 48., 49.,
       49., 50., 50., 51., 51., 52., 52., 53., 53., 54., 54., 55., 55.,
       55., 56., 56., 57., 57., 58., 58., 59., 59., 59., 60., 60., 61.,
       61., 62., 62., 63., 63., 63., 64.])

In [19]:
print(len(spike_channel_id[spike_total_cnt > 0]))
print(len(channel_id_pkl))

129
137


In [ ]:
# data["meta"]['spikeID'].shape
from neuro_data_analysis.PL2_reading_lib import read_channel_unit_info, pl2_info
local_PL2_root = r"S:\Data-Ephys-Raw"
network_PL2_root = r"N:\Data-Ephys-Raw"
for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):   
    PL2_path = join(local_PL2_root, f"{exprow.ephysFN}.pl2")
    if not os.path.exists(PL2_path):
        PL2_path = PL2_path.replace("S:", "N:") # use the network drive for non local PL2 files
        if not os.path.exists(PL2_path):
            # print(f"PL2 file does not exist on local or network drive: {PL2_path}")
            continue
    spike_channel_id, spike_unit_id, spike_total_cnt = read_channel_unit_info(PL2_path)
    pkldata = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    if "spikeID" in pkldata["meta"].keys():
        length_match = len(pkldata["meta"]["spikeID"][0]) == len(spike_channel_id)
        exact_match = (pkldata["meta"]["spikeID"][0].astype(int) == spike_channel_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Spike ID mismatch: {pkldata['meta']['spikeID']} [len: {len(pkldata['meta']['spikeID'][0])}] != {spike_channel_id} [len: {len(spike_channel_id)}]")
    # else:
    #     pkldata["meta"]["spikeID"] = spike_channel_id[None, :]
        
    if "unitID" in pkldata["meta"].keys():
        length_match = len(pkldata["meta"]["unitID"][0]) == len(spike_unit_id)
        exact_match = (pkldata["meta"]["unitID"][0].astype(int) == spike_unit_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Unit ID mismatch: {pkldata['meta']['unitID']} [len: {len(pkldata['meta']['unitID'][0])}] != {spike_unit_id} [len: {len(spike_unit_id)}]")
    # else:
    #     pkldata["meta"]["unitID"] = spike_unit_id[None, :]
    # pkldata["meta"]["total_spike_cnt"] = spike_total_cnt
    # pkl.dump(pkldata, open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "wb"))



68it [00:06,  9.16it/s]C:\Users\ponce\AppData\Local\Temp\ipykernel_58096\883216326.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  exact_match = (pkldata["meta"]["spikeID"][0].astype(int) == spike_channel_id)
C:\Users\ponce\AppData\Local\Temp\ipykernel_58096\883216326.py:24: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  exact_match = (pkldata["meta"]["unitID"][0].astype(int) == spike_unit_id)
90it [00:06, 24.35it/s]

Alfa-12032021-004 Spike ID mismatch: [[ 1.  1.  2.  2.  3.  3.  4.  5.  5.  5.  6.  6.  7.  8.  8.  8.  9.  9.
  10. 11. 12. 13. 14. 15. 16. 17. 18. 18. 19. 19. 20. 20. 21. 22. 23. 24.
  24. 25. 25. 26. 27. 27. 28. 28. 29. 30. 31. 32. 32. 32. 33. 33. 34. 34.
  34. 35. 35. 36. 37. 37. 38. 39. 39. 40. 40. 41. 41. 42. 42. 43. 43. 43.
  44. 45. 46. 47. 48. 49. 50. 50. 51. 52. 53. 53. 54. 55. 55. 56. 57. 58.
  58. 58. 58. 59. 59. 60. 61. 62. 63. 63. 64. 64.]] [len: 102] != [ 1  1  2  2  3  4  5  5  5  6  7  8  8  8  9  9 10 11 12 13 14 15 16 17
 18 19 20 21 22 23 24 25 26 27 28 28 29 30 31 32 32 32 33 33 34 34 34 35
 35 36 37 37 38 39 39 40 40 41 41 42 42 43 43 43 44 45 46 47 48 49 50 50
 51 52 53 53 54 55 55 56 57 58 58 58 58 59 60 61 62 63 63 64 64] [len: 93]
Alfa-12032021-004 Unit ID mismatch: [[0. 1. 0. 1. 0. 1. 1. 0. 1. 2. 0. 1. 1. 0. 1. 2. 0. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.
  1. 2. 0. 1. 0. 1. 2. 0. 1. 1. 0. 1. 1. 0. 1. 0.

107it [00:07, 14.09it/s]

Alfa-14052021-003 Spike ID mismatch: [[ 1.  1.  2.  2.  3.  3.  4.  4.  5.  5.  6.  6.  7.  7.  8.  8.  9.  9.
  10. 10. 10. 11. 11. 12. 12. 13. 13. 14. 14. 15. 15. 16. 16. 17. 17. 18.
  18. 19. 19. 20. 20. 21. 21. 22. 22. 23. 23. 24. 24. 24. 25. 25. 26. 26.
  27. 27. 28. 28. 29. 29. 30. 30. 31. 31. 32. 32. 32. 33. 33. 34. 34. 35.
  35. 35. 36. 36. 37. 37. 38. 38. 39. 39. 40. 40. 41. 41. 41. 42. 42. 43.
  43. 44. 44. 45. 45. 45. 46. 46. 47. 47. 48. 48. 49. 49. 50. 50. 51. 51.
  52. 52. 53. 53. 54. 54. 55. 55. 56. 56. 57. 57. 58. 58. 58. 59. 59. 60.
  60. 61. 61. 61. 62. 62. 63. 63. 64. 64.]] [len: 136] != [ 1  1  2  2  3  3  4  5  5  6  6  7  7  8  8  9  9 10 10 10 11 12 12 13
 13 14 14 15 15 16 17 18 18 19 19 20 21 21 22 23 23 24 24 25 26 26 27 27
 28 28 29 29 30 30 31 32 32 32 33 33 34 34 35 35 35 36 36 37 37 38 38 39
 39 40 40 41 41 41 42 42 43 43 44 44 45 45 45 46 46 47 47 48 48 49 49 50
 50 51 51 52 52 53 53 54 54 55 55 56 56 57 57 58 58 58 59 59 60 60 61 61
 61 62 62 63 63 64 64]

In [ ]:
print(len(pkldata["meta"]["spikeID"][0]))
print(len(spike_channel_id))


102
93


In [ ]:
exprow.ephysFN

'Alfa-12032021-004'

C:\Users\ponce\AppData\Local\Temp\ipykernel_58096\2089072093.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  (pkldata["meta"]["spikeID"][0].astype(int) == spike_channel_id).all()


AttributeError: 'bool' object has no attribute 'all'

In [ ]:
pkldata["meta"]["spikeID"]

array([[ 1.,  1.,  2.,  2.,  3.,  3.,  4.,  5.,  5.,  5.,  6.,  6.,  7.,
         8.,  8.,  8.,  9.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
        18., 18., 19., 19., 20., 20., 21., 22., 23., 24., 24., 25., 25.,
        26., 27., 27., 28., 28., 29., 30., 31., 32., 32., 32., 33., 33.,
        34., 34., 34., 35., 35., 36., 37., 37., 38., 39., 39., 40., 40.,
        41., 41., 42., 42., 43., 43., 43., 44., 45., 46., 47., 48., 49.,
        50., 50., 51., 52., 53., 53., 54., 55., 55., 56., 57., 58., 58.,
        58., 58., 59., 59., 60., 61., 62., 63., 63., 64., 64.]])

In [ ]:
for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    data = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    # rasters = data["rasters"]
    meta = data["meta"]
    # Trials = data["Trials"]
    data["meta"]

Caos-12022024-003 Caos Exp 1
Caos-12022024-006 Caos Exp 2
Caos-12042024-003 Caos Exp 3
Caos-12042024-007 Caos Exp 4
Caos-12092024-004 Caos Exp 5
Caos-12102024-003 Caos Exp 6
Caos-12102024-006 Caos Exp 7
Diablito-03122024-003 Diablito Exp 1
Diablito-11122024-003 Diablito Exp 2
Diablito-11122024-005 Diablito Exp 3
Diablito-12122024-004 Diablito Exp 4
Diablito-12122024-006 Diablito Exp 5
Alfa-04082020-003 Alfa Exp 1
Alfa-06082020-004 Alfa Exp 2
Alfa-10082020-004 Alfa Exp 2
Alfa-12082020-004 Alfa Exp 3
Alfa-14082020-003 Alfa Exp 4
Alfa-25082020-007 Alfa Exp 5
Alfa-27082020-003 Alfa Exp 6
Alfa-01092020-004 Alfa Exp 7
Alfa-02092020-004 Alfa Exp 8
Alfa-02092020-006 Alfa Exp 9
Alfa-08092020-004 Alfa Exp 10
Alfa-11092020-003 Alfa Exp 11
Alfa-11092020-006 Alfa Exp 12
Alfa-16092020-003 Alfa Exp 13
Alfa-16092020-005 Alfa Exp 14
Alfa-27102020-007 Alfa Exp 15
Alfa-27102020-008 Alfa Exp 15
Alfa-27102020-009 Alfa Exp 15
Alfa-04112020-005 Alfa Exp 16
Alfa-06112020-006 Alfa Exp 17
Alfa-12012021-003 Alfa